<H1> Example on using the datasets for quantum characterization and control </H1>

In [20]:
# preample
import zipfile
import pickle
import os
import numpy as np

In [21]:
'''The datalist below lists each different dataset category.
Each category comprises two compressed files:
(a) one for the non-distorted examples; and
(b) another for the distorted examples.

Each dataset is stored in a folder corresponding to its category on Cloudstor.
There are 52 datasets in the QDataSet and 26 categories.


To run the code:
1. Download both the distorted and non-distorted datasets from Cloudstor
(assuming both are to be used);
2. To select a dataset for the model (e.g. G_1q_X), uncomment the dataset
in the dataset list below.'''

'The datalist below lists each different dataset category.\nEach category comprises two compressed files:\n(a) one for the non-distorted examples; and\n(b) another for the distorted examples.\n\nEach dataset is stored in a folder corresponding to its category on Cloudstor.\nThere are 52 datasets in the QDataSet and 26 categories.\n\n\nTo run the code:\n1. Download both the distorted and non-distorted datasets from Cloudstor\n(assuming both are to be used);\n2. To select a dataset for the model (e.g. G_1q_X), uncomment the dataset\nin the dataset list below.'

In [22]:
datalist = [
'G_1q_X',
# 'G_1q_XY',
# 'G_1q_XY_XZ_N1N5',
# 'G_1q_XY_XZ_N1N6',
# 'G_1q_XY_XZ_N3N6',
# 'G_1q_X_Z_N1',
# 'G_1q_X_Z_N2',
# 'G_1q_X_Z_N3',
# 'G_1q_X_Z_N4',
# 'G_2q_IX-XI_IZ-ZI_N1-N6',
# 'G_2q_IX-XI-XX',
# 'G_2q_IX-XI-XX_IZ-ZI_N1-N5',
# 'G_2q_IX-XI-XX_IZ-ZI_N1-N5',
# 'S_1q_X',
# 'S_1q_XY',
# 'S_1q_XY_XZ_N1N5',
# 'S_1q_XY_XZ_N1N6',
# 'S_1q_XY_XZ_N3N6',
# 'S_1q_X_Z_N1',
# 'S_1q_X_Z_N2',
# 'S_1q_X_Z_N3',
# 'S_1q_X_Z_N4',
# 'S_2q_IX-XI_IZ-ZI_N1-N6',
# 'S_2q_IX-XI-XX',
# 'S_2q_IX-XI-XX_IZ-ZI_N1-N5',
# 'S_2q_IX-XI-XX_IZ-ZI_N1-N6',
]

In [23]:
'''Create two strings, one for each of the undistorted and distorted datasets.'''
data1 = datalist[0]
data2 = data1 + '_D'

In [24]:
'''Check strings created correction'''
print(data1)
print(data2)

G_1q_X
G_1q_X_D


In [26]:
'''Set the working directory to the location of the datasets, example below.'''
#pirnt current working directory
print(os.getcwd())
print(os.listdir())
os.chdir(f'./{data1}')

/mnt/c/Users/ChrisWiseXPSLocal/GitHub/QDataSet/QuantumDS
['.owncloudsync.log', 'G_1q_X']


## Characterization of an open quantum system

In [27]:
'''Define a function to extract the data from the dataset.'''

def load_dataset(dataset_name, num_training_ex, num_testing_ex):
    
    # initialize empt lists for storing the data
    data_input         = []
    data_target        = []
    
    # 1) unzip the dataset zipfile
    fzip               = zipfile.ZipFile("%s.zip"%dataset_name, mode='r')
    
    # loop over example files
    #########################################################
    for idx_ex in range(num_training_ex + num_testing_ex):
    
        # 2) extract the example file from the dataset 
        fname = "%s_ex_%d"%(dataset_name, idx_ex)
        fzip.extract( fname )
    
        # 3) load the example file
        f     = open(fname,  "rb")
        data  = pickle.load(f)
    
        # 4) extract the useful information from the example file:
        
        # For noise spectroscopy, we need to extract a set of control pulse and the 
        # correpsonding informationally-complete observables
        
        # add the pair of input and target to the corresponding lists
        data_input.append( data["pulses"][0:1, :, 0, :])
        data_target.append( data["expectations"] )
        
        # 5) close and delete the example file
        f.close()
        os.remove(fname)
    #########################################################
    # 5) close the dataset zipfile
    fzip.close()
    
    # 6) split the data into training and testing
    data_input            = np.concatenate(data_input, axis=0)
    data_target           = np.concatenate(data_target, axis=0)
    
    training_input        = data_input[0:num_training_ex, :]
    testing_input         = data_input[num_training_ex:num_training_ex+num_testing_ex, :]
    
    training_target       = data_target[0:num_training_ex, :]
    testing_target        = data_target[num_training_ex:num_training_ex+num_testing_ex, :]
    
    return training_input, training_target, testing_input, testing_target

In [28]:
'''Define the dataset parameters'''
dataset_name    = data1#"G_1q_XY_XZ_N1N5" # dataset name
num_training_ex = 7                 # number of training examples
num_testing_ex  = 3                 # number of testing examples

In [29]:
# load the dataset
training_input, training_target, testing_input, testing_target = load_dataset(dataset_name, num_training_ex, num_testing_ex)

In [30]:
'''Inspect the shape of the training and test dataframes (if need be).'''
print(training_input.shape, training_target.shape, testing_input.shape, testing_target.shape)

(7, 1024, 1) (7, 18) (3, 1024, 1) (3, 18)


In [31]:
'''Set input parameter shape based on number of axes along which controls / noise is applied.'''
axnum = training_input.shape[2]
axnum2 = training_target.shape[1]
print(axnum, axnum2)

1 18


In [33]:
# use the dataset for training and testing an ML algorithm

#############################################
##          PUT YOUR CODE HERE             ##

# trained_model  = my_training_function(training_input, training_target)

# performance    = my_testing_function(trained_model, testing_input, testing_target)

# noise_spectrum = estimate_noise(trained_model)

### Below is an example using tensorflow  ###

import tensorflow.keras as K

input_layer   = K.Input(shape=(None, axnum))

output_layer  = K.layers.LSTM(axnum2, return_sequences=False)(input_layer)

ml_model      = K.Model(input_layer, output_layer)

ml_model.compile(optimizer=K.optimizers.Adam(), loss='mse')

ml_model.fit(training_input, training_target, epochs=10, validation_data = (testing_input, testing_target))

2023-02-07 17:15:50.800789: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-07 17:15:57.130117: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-07 17:15:57.130245: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-07 17:16:30.473508: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 0.3332 - val_loss: 0.3332
Epoch 2/10
1/1 [==============================] - 1s 564ms/step - loss: 0.3330 - val_loss: 0.3332
Epoch 3/10
1/1 [==============================] - 1s 503ms/step - loss: 0.3329 - val_loss: 0.3331
Epoch 4/10
1/1 [==============================] - 1s 749ms/step - loss: 0.3327 - val_loss: 0.3331
Epoch 5/10
1/1 [==============================] - 1s 668ms/step - loss: 0.3326 - val_loss: 0.3331
Epoch 6/10
1/1 [==============================] - 1s 778ms/step - loss: 0.3324 - val_loss: 0.3330
Epoch 7/10
1/1 [==============================] - 1s 677ms/step - loss: 0.3322 - val_loss: 0.3330
Epoch 8/10
1/1 [==============================] - 1s 805ms/step - loss: 0.3321 - val_loss: 0.3329
Epoch 9/10
1/1 [==============================] - 1s 711ms/step - loss: 0.3319 - val_loss: 0.3328
Epoch 10/10
1/1 [==============================] - 1s 646ms/step - loss: 0.3317 - val_loss: 0.3327


## Using the $V_O$ operators in a calculation

In [34]:
'''Define a function to extract the data from the dataset.'''
def load_Vo_dataset(dataset_name, num_ex):
    
    # initialize empt lists for storing the data
    pulses         = []
    VX             = []
    VY             = []
    VZ             = []
    # 1) unzip the dataset zipfile
    fzip               = zipfile.ZipFile("%s.zip"%dataset_name, mode='r')
    
    # loop over example files
    #########################################################
    for idx_ex in range(num_ex):
    
        # 2) extract the example file from the dataset 
        fname = "%s_ex_%d"%(dataset_name, idx_ex)
        fzip.extract( fname )
    
        # 3) load the example file
        f     = open(fname,  "rb")
        data  = pickle.load(f)
    
        # 4) extract the useful information from the example file:
        
        # For noise spectroscopy, we need to extract a set of control pulse and the 
        # correpsonding informationally-complete observables
        
        # add the pair of input and target to the corresponding lists
        pulses.append( data["pulses"][0:1, :, 0, :])
        VX.append( data["Vo_operator"][0] )
        VY.append( data["Vo_operator"][1] )
        VZ.append( data["Vo_operator"][2] )
        
        # 5) close and delete the example file
        f.close()
        os.remove(fname)
    #########################################################
    # 5) close the dataset zipfile
    fzip.close()
    
    # 6) split the data into training and testing
    pulses            = np.concatenate(pulses, axis=0)
    VX                = np.concatenate(VX, axis=0)
    VY                = np.concatenate(VY, axis=0)
    VZ                = np.concatenate(VZ, axis=0)   
    
    return pulses, VX, VY, VZ

In [35]:
# define the dataset parameters
dataset_name    = data2 #"G_1q_XY_XZ_N1N5_D" # dataset name
num_ex = 7                            # number of examples

In [36]:
pulses, VX, VY, VZ = load_Vo_dataset(dataset_name, num_ex)

In [37]:
print(pulses.shape, VX.shape, VY.shape, VZ.shape)

(7, 1024, 1) (7, 2, 2) (7, 2, 2) (7, 2, 2)


In [38]:
'''Set input parameter shape based on number of qubits and set corresponding shape of flattened 
tensor for input into LSTM parameter.'''
# axnum = training_input.shape[2]
axnum3 = VX.shape[2]
lstmflat = axnum3 **2


In [39]:
# use the Vo in a calculation

#############################################
##          PUT YOUR CODE HERE             ##

# result = my_calculation(pulses, VX)

In [40]:
# train an ML model to learn Vo

#############################################
##          PUT YOUR CODE HERE             ##

# trained_model = my_training_function(training_input=pulses[0:100,;], training_target=VX[0:100, :])

# performance   = my_testing_function(trained_model, testing_input=pulses[100:,:], testing_target=Vx[100:,:])

### Below is an example using tensorflow  ###

import tensorflow.keras as K

input_layer   = K.Input(shape=(None, axnum))

output_layer  = K.layers.Reshape((axnum3,axnum3)) (  K.layers.LSTM(lstmflat, return_sequences=False)(input_layer) )

ml_model      = K.Model(input_layer, output_layer)

ml_model.compile(optimizer=K.optimizers.Adam(), loss='mse')

ml_model.fit(pulses, VX, epochs=10, validation_split=0.1)

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 0.4999 - val_loss: 0.4986
Epoch 2/10
1/1 [==============================] - 0s 435ms/step - loss: 0.4986 - val_loss: 0.4972
Epoch 3/10
1/1 [==============================] - 0s 475ms/step - loss: 0.4972 - val_loss: 0.4959
Epoch 4/10
1/1 [==============================] - 1s 517ms/step - loss: 0.4959 - val_loss: 0.4946
Epoch 5/10
1/1 [==============================] - 0s 491ms/step - loss: 0.4946 - val_loss: 0.4932
Epoch 6/10
1/1 [==============================] - 0s 472ms/step - loss: 0.4932 - val_loss: 0.4919
Epoch 7/10
1/1 [==============================] - 0s 486ms/step - loss: 0.4919 - val_loss: 0.4905
Epoch 8/10
1/1 [==============================] - 1s 504ms/step - loss: 0.4905 - val_loss: 0.4891
Epoch 9/10
1/1 [==============================] - 1s 517ms/step - loss: 0.4891 - val_loss: 0.4878
Epoch 10/10
1/1 [==============================] - 1s 526ms/step - loss: 0.4878 - val_loss: 0.4864


## Model the effect of control distortions

In [41]:
# define the dataset parameters
dataset_name    = data2 #"G_1q_XY_XZ_N1N5_D" # dataset name
num_training_ex = 7                   # number of training examples
num_testing_ex  = 3                   # number of testing examples

In [42]:
# load the dataset
training_input, training_target, testing_input, testing_target = load_dataset(dataset_name, num_training_ex, num_testing_ex)

In [43]:
print(training_input.shape, training_target.shape, testing_input.shape, testing_target.shape)

(7, 1024, 1) (7, 18) (3, 1024, 1) (3, 18)


In [44]:
'''Set shape parameters based on shape of inputs and outputs.'''
axnum = training_input.shape[2]
lstmout = training_target.shape[1]


In [45]:
# use the dataset for training and testing an ML algorithm

#############################################
##          PUT YOUR CODE HERE             ##

# trained_model = my_training_function(training_input, training_target)

# performance   = my_testing_function(trained_model, testing_input, testing_target)

### Below is an example using tensorflow  ###

import tensorflow.keras as K

input_layer   = K.Input(shape=(None, axnum))

output_layer  = K.layers.LSTM(lstmout, return_sequences=False)(input_layer)

ml_model      = K.Model(input_layer, output_layer)

ml_model.compile(optimizer=K.optimizers.Adam(), loss='mse')

ml_model.fit(training_input, training_target, epochs=10, validation_data = (testing_input, testing_target))

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 0.3332 - val_loss: 0.3336
Epoch 2/10
1/1 [==============================] - 1s 643ms/step - loss: 0.3328 - val_loss: 0.3341
Epoch 3/10
1/1 [==============================] - 1s 604ms/step - loss: 0.3323 - val_loss: 0.3345
Epoch 4/10
1/1 [==============================] - 1s 687ms/step - loss: 0.3318 - val_loss: 0.3351
Epoch 5/10
1/1 [==============================] - 1s 579ms/step - loss: 0.3313 - val_loss: 0.3356
Epoch 6/10
1/1 [==============================] - 1s 628ms/step - loss: 0.3308 - val_loss: 0.3363
Epoch 7/10
1/1 [==============================] - 1s 602ms/step - loss: 0.3303 - val_loss: 0.3370
Epoch 8/10
1/1 [==============================] - 1s 596ms/step - loss: 0.3297 - val_loss: 0.3377
Epoch 9/10
1/1 [==============================] - 1s 601ms/step - loss: 0.3291 - val_loss: 0.3386
Epoch 10/10
1/1 [==============================] - 1s 611ms/step - loss: 0.3284 - val_loss: 0.3397


## Learning a controller for a quantum system

In [46]:
# define the dataset parameters
dataset_name    = data1 #"G_1q_XY_XZ_N1N5"   # dataset name
num_training_ex = 7                   # number of training examples
num_testing_ex  = 3                   # number of testing examples

In [47]:
# load the dataset [inputs and targets are echanged for quantum control problem, this is the inverse of the 
# characterization problem]
training_target, training_input, testing_target, testing_input  = load_dataset(dataset_name, num_training_ex, num_testing_ex)

In [48]:
print(training_input.shape, training_target.shape, testing_input.shape, testing_target.shape)



(7, 18) (7, 1024, 1) (3, 18) (3, 1024, 1)


In [49]:
'''Set shape parameters based on shape of inputs and outputs.'''
axnum4 = training_input.shape[1]
axnum5 = training_target.shape[1]
axnum6 = training_target.shape[2]
axnum7 = axnum5 * axnum6

In [50]:
# use the dataset for training and testing an ML algorithm

#############################################
##          PUT YOUR CODE HERE             ##

# trained_model = my_training_function(training_input, training_target)

# performance   = my_testing_function(trained_model, testing_input, testing_target)

### Below is an example using tensorflow  ###

import tensorflow.keras as K

input_layer   = K.Input(shape=(axnum4))

output_layer  = K.layers.Reshape((axnum5,axnum6))( K.layers.Dense(axnum7)(input_layer) )

ml_model      = K.Model(input_layer, output_layer)

ml_model.compile(optimizer=K.optimizers.Adam(), loss='mse')

ml_model.fit(training_input, training_target, epochs=10, validation_data = (testing_input, testing_target))

Epoch 1/10
1/1 [==============================] - 0s 497ms/step - loss: 439.1874 - val_loss: 523.2390
Epoch 2/10
1/1 [==============================] - 0s 37ms/step - loss: 439.0955 - val_loss: 523.2410
Epoch 3/10
1/1 [==============================] - 0s 36ms/step - loss: 439.0036 - val_loss: 523.2431
Epoch 4/10
1/1 [==============================] - 0s 38ms/step - loss: 438.9117 - val_loss: 523.2453
Epoch 5/10
1/1 [==============================] - 0s 35ms/step - loss: 438.8198 - val_loss: 523.2474
Epoch 6/10
1/1 [==============================] - 0s 36ms/step - loss: 438.7280 - val_loss: 523.2495
Epoch 7/10
1/1 [==============================] - 0s 39ms/step - loss: 438.6362 - val_loss: 523.2516
Epoch 8/10
1/1 [==============================] - 0s 42ms/step - loss: 438.5445 - val_loss: 523.2538
Epoch 9/10
1/1 [==============================] - 0s 42ms/step - loss: 438.4527 - val_loss: 523.2559
Epoch 10/10
1/1 [==============================] - 0s 37ms/step - loss: 438.3611 - val_los